In [30]:
import sqlparse

def extract_columns_and_conditions_per_line(sql_query):
    def get_column_names(token):
        if isinstance(token, sqlparse.sql.IdentifierList):
            return [str(ident) for ident in token.get_identifiers()]
        return [str(token)]
    
    def process_condition(token):
        column_names_condition = []
        for tok in token.tokens:
            if isinstance(tok, sqlparse.sql.IdentifierList):
                column_names_condition.extend(get_column_names(tok))
            elif isinstance(tok, sqlparse.sql.Identifier):
                column_names_condition.extend(get_column_names(tok))
        return column_names_condition

    def process_line(line):
        column_names = []
        parsed_line = sqlparse.parse(line)[0]
        for token in parsed_line.tokens:
            if isinstance(token, sqlparse.sql.Where):
                column_names.extend(process_condition(token))
            elif isinstance(token, sqlparse.sql.IdentifierList) and 'SELECT' in line.upper():
                column_names.extend(get_column_names(token))
            elif isinstance(token, sqlparse.sql.Identifier):
                column_names.extend(get_column_names(token))
            
        return column_names

    lines = sql_query.splitlines()
    result = []
    for line in lines:
        column_names= process_line(line)
        result += column_names
    return result

def queries(d):
  if type(d) != sqlparse.sql.Token:
     paren = isinstance(d, sqlparse.sql.Parenthesis)
     v = [queries(i) for i in (d if not paren else d[1:-1])]
     subseq, qrs = ''.join(str(i[0]) for i in v), [x for _, y in v for x in y]
     if [*d][paren].value == 'SELECT':
        return '<>', [subseq]+qrs
     return subseq, qrs
  return d, []

In [44]:
def select_sql_column(query):
    _, subqueries = queries(sqlparse.parse(query)[0])
    choose_col = set()
    print
    for query in subqueries:
        results = set(extract_columns_and_conditions_per_line(query))
        choose_col.update(results)
    return list(choose_col)

In [31]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

desc_col = {}

description_df = pd.read_excel('src/New_query_Description.xlsx',header=1)
description_df = description_df[['Column','Description']].dropna()
x = 0
for i, row in description_df.iterrows():
    desc_col[x] = row['Column']
    x += 1

In [32]:
descriptions = description_df['Description'].tolist()

In [33]:
pointx_cols = pd.read_csv('src/pointx_fbs_rpt_dly.csv').columns.to_list()
assert len(pointx_cols) == len(desc_col), "Length of columns in Pointx table and Descriptions are not equal"

In [41]:
compare_df = pd.read_csv('src/compare_result.csv')
questions = compare_df['Question'].to_list()
sql_queries = compare_df['SQL'].to_list()

In [42]:
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_col_max_score(question,description_df=description_df,model=model):
    q_emb = model.encode(question)
    scores = [float(util.cos_sim(q_emb, model.encode(des))) for des in description_df['Description'].to_list()]
    print(np.mean(scores),np.max(scores))
    print("COLUMN:",description_df.iloc[np.argmax(scores)]['Column'])
    print("DESC:",description_df.iloc[np.argmax(scores)]['Description'],'\n')
    
    return int(np.argmax(scores))

In [45]:
for i, q in enumerate(questions[:10]):
    print(q)
    print('LABEL COLUMNS:',select_sql_column(sql_queries[i]))
    get_col_max_score(q)

How many daily active users each day?
SELECT event_date, COUNT(DISTINCT user_pseudo_id)
FROM (
    SELECT event_date, ga_session_id, user_pseudo_id
    FROM team_tds.tds_intern.pointx_fbs_txn_rpt_dly
    GROUP BY event_date, ga_session_id, user_pseudo_id
    HAVING SUM(engagement_time_msec) > 10*1000
)
GROUP BY event_date
ORDER BY event_date ASC


IndexError: tuple index out of range

In [25]:
import sqlparse

def extract_columns_and_conditions_per_line(sql_query):
    def get_column_names(token):
        if isinstance(token, sqlparse.sql.IdentifierList):
            return [str(ident) for ident in token.get_identifiers()]
        return [str(token)]
    
    def process_condition(token):
        column_names_condition = []
        for tok in token.tokens:
            if isinstance(tok, sqlparse.sql.IdentifierList):
                column_names_condition.extend(get_column_names(tok))
            elif isinstance(tok, sqlparse.sql.Identifier):
                column_names_condition.extend(get_column_names(tok))
        return column_names_condition

    def process_line(line):
        column_names = []
        parsed_line = sqlparse.parse(line)[0]
        for token in parsed_line.tokens:
            if isinstance(token, sqlparse.sql.Where):
                column_names.extend(process_condition(token))
            elif isinstance(token, sqlparse.sql.IdentifierList) and 'SELECT' in line.upper():
                column_names.extend(get_column_names(token))
            elif isinstance(token, sqlparse.sql.Identifier):
                column_names.extend(get_column_names(token))
            
        return column_names

    lines = sql_query.splitlines()
    result = []
    for line in lines:
        column_names= process_line(line)
        result += column_names
    return result

def queries(d):
  if type(d) != sqlparse.sql.Token:
     paren = isinstance(d, sqlparse.sql.Parenthesis)
     v = [queries(i) for i in (d if not paren else d[1:-1])]
     subseq, qrs = ''.join(str(i[0]) for i in v), [x for _, y in v for x in y]
     if [*d][paren].value == 'SELECT':
        return '<>', [subseq]+qrs
     return subseq, qrs
  return d, []


# Example SQL query with multiple lines
s = """SELECT name, email
     FROM user
     WHERE email IS NOT NULL
     AND id IN (SELECT cID FROM customer WHERE points > 5)
"""


{'cID', 'customer', 'email', 'id', 'name'}

In [20]:
import sqlparse
def queries(d):
  if type(d) != sqlparse.sql.Token:
     paren = isinstance(d, sqlparse.sql.Parenthesis)
     v = [queries(i) for i in (d if not paren else d[1:-1])]
     subseq, qrs = ''.join(str(i[0]) for i in v), [x for _, y in v for x in y]
     if [*d][paren].value == 'SELECT':
        return '<subquery>', [subseq]+qrs
     return subseq, qrs
  return d, []

s="""SELECT name, email
     FROM (SELECT * FROM user WHERE email IS NOT NULL)
     WHERE id IN (SELECT cID FROM customer WHERE points > 5)
"""

_, subqueries = queries(sqlparse.parse(s)[0])

subqueries

['SELECT name, email\n     FROM <subquery>\n     WHERE id IN <subquery>\n',
 'SELECT * FROM user WHERE email IS NOT NULL',
 'SELECT cID FROM customer WHERE points > 5']

In [19]:
for q in subqueries:
    result = extract_columns_and_conditions_per_line(q)
    print(result)

[{'ColumnNamesCondition': ['name', 'email']}, {'ColumnNamesCondition': ['subquery']}, {'ColumnNamesCondition': ['id', 'subquery']}]
[{'ColumnNamesCondition': ['email']}]
[{'ColumnNamesCondition': ['cID', 'customer']}]


Cosine-Similarity: tensor([[0.8958]])
